In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import IPython.display as ipd
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
  SAMPLING_RATE = 16000
  DURATION = 10
  DATA_PER_CLASS = 60*40*SAMPLING_RATE

In [ ]:
A, _ = librosa.load('/content/drive/MyDrive/val_set/speech_and_music/speech_indian.wav', sr=SAMPLING_RATE)
A = A[:DATA_PER_CLASS]

In [ ]:
B, _ = librosa.load('/content/drive/MyDrive/val_set/speech_and_music/music_long.wav', sr=SAMPLING_RATE)
B = B[:DATA_PER_CLASS]
print(B.shape)

(38400000,)


In [ ]:
C = np.zeros((DATA_PER_CLASS))

In [ ]:
noise = np.random.rand((DURATION*SAMPLING_RATE))
noise = noise*2
noise = noise - 1

In [ ]:
SPEECH_SIZE = A.shape[0]
endpt = (DURATION*SAMPLING_RATE)*int(SPEECH_SIZE/(DURATION*SAMPLING_RATE))
X_speech = A[:endpt]
X_speech = np.resize(X_speech, (-1, DURATION*SAMPLING_RATE))
y_speech = np.array(['speech']*len(X_speech))
print(X_speech.shape)

(239, 160000)


In [ ]:
MUSIC_SIZE = B.shape[0]
endpt = (DURATION*SAMPLING_RATE)*int(MUSIC_SIZE/(DURATION*SAMPLING_RATE))
X_music = B[:endpt]
X_music = np.resize(X_music, (-1, DURATION*SAMPLING_RATE))
y_music = np.array(['music']*len(X_music))
print(X_music.shape)

(239, 160000)


In [ ]:
SILENCE_SIZE = C.shape[0]
endpt = (DURATION*SAMPLING_RATE)*int(SILENCE_SIZE/(DURATION*SAMPLING_RATE))
X_silence = C[:endpt]
X_silence = np.resize(X_silence, (-1, DURATION*SAMPLING_RATE))
y_silence = np.array(['silence']*len(X_silence))
print(X_silence.shape)

(239, 160000)


In [ ]:
# Randomly add white noise
X = np.concatenate([X_speech, X_music, X_silence])
y = np.concatenate([y_speech, y_music, y_silence])

for i in range(X.shape[0]):
    # Adding 0 to 10 percent noise randomly to each sound
    X[i] = X[i] + noise*(np.random.randint(10)/100)


In [ ]:
from sklearn.utils import shuffle
X, y = shuffle(X, y)

In [ ]:
np.save('/content/drive/MyDrive/val_set/processed_data/X', X)

In [ ]:
np.save('/content/drive/MyDrive/val_set/processed_data/y', y)

In [ ]:
X = np.load('/content/drive/MyDrive/val_set/processed_data/X.npy')

In [ ]:
def calc_spec(x):
    n_fft = 1024
    hop_length = 512
    win_length = 1024
    X = np.abs(librosa.stft(x, n_fft = n_fft, hop_length = hop_length, win_length = win_length, window='hann', dtype = np.complex256))
    X = librosa.power_to_db(X**2,ref=np.max)
    return X

In [ ]:
from tqdm import tqdm

In [ ]:
def convert2spect(X):
  X_spectrograms = []
  for i in tqdm(X):
    X_spectrograms.append(calc_spec(i))
  return np.array(X_spectrograms)

In [ ]:
A = convert2spect(X)
print(A.shape)

100%|██████████| 717/717 [00:22<00:00, 32.43it/s]


(717, 513, 313)


In [ ]:
np.save('/content/drive/MyDrive/val_set/processed_data/combined_spectrogram.npy', A)

In [ ]:
print(X.shape)

(717, 160000)
